# Prerequisites

Install the necessary Python modules to use and interact with [NVIDIA Triton Inference Server](https://github.com/triton-inference-server/server/).

In [ ]:
! pip install torch==1.10.0 sagemaker transformers==4.9.1 tritonclient[all]

# Part 1 - Setup

In [ ]:
import argparse
import boto3
import copy
import datetime
import json
import numpy as np
import os
import pandas as pd
import pprint
import re
import sagemaker
import sys
import time
from time import gmtime, strftime
import tritonclient.http as http_client

In [ ]:
session = boto3.Session()
role = sagemaker.get_execution_role()

sm_client = session.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=session)
sm_runtime_client = boto3.client("sagemaker-runtime")

region = boto3.Session().region_name

In [ ]:
account_id_map = {
    "us-east-1": "785573368785",
    "us-east-2": "007439368137",
    "us-west-1": "710691900526",
    "us-west-2": "301217895009",
    "eu-west-1": "802834080501",
    "eu-west-2": "205493899709",
    "eu-west-3": "254080097072",
    "eu-north-1": "601324751636",
    "eu-south-1": "966458181534",
    "eu-central-1": "746233611703",
    "ap-east-1": "110948597952",
    "ap-south-1": "763008648453",
    "ap-northeast-1": "941853720454",
    "ap-northeast-2": "151534178276",
    "ap-southeast-1": "324986816169",
    "ap-southeast-2": "355873309152",
    "cn-northwest-1": "474822919863",
    "cn-north-1": "472730292857",
    "sa-east-1": "756306329178",
    "ca-central-1": "464438896020",
    "me-south-1": "836785723513",
    "af-south-1": "774647643957",
}

***

# Part 2 - Generate TensorRT Model

In the following cells, we are using [HuggingFace Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto) to load a pre-trained model from the [HuggingFace Model Hub](https://huggingface.co/models). We then convert the model to the ONNX format, and compile it using NVIDIA TensorRT - namely its command-line wrapper tool, `trtexec` -, using the scripts provided in the official AWS Sample for [SageMaker Triton](https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-triton).

NVIDIA TensorRT is an SDK that facilitates high-performance machine learning inference. You can use it to create `engines` from models that have already been trained, 
optimizing for a selected GPU architecture. Triton natively supports the TensorRT runtime, which enables you to easily deploy a TensorRT engine and pair it with the rich features that Triton provides.

### Parameters:

* `model_name`: Model identifier from the Hugging Face model hub library

In [ ]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"

## Option 1 - TensorRT Model with Amazon SageMaker Studio

> **WARNING**: The next cell will only work if you have first created a custom Studio image, described in Step 2 of this repository's README. Change the `RUNNING_IN_STUDIO` to `True` if this is the case.

In [ ]:
RUNNING_IN_STUDIO = False

In [ ]:
if RUNNING_IN_STUDIO:
    !/bin/bash ./workspace/generate_model_trt.sh $model_id && rm -rf ensemble_hf/bert-trt/1 && mkdir -p ensemble_hf/bert-trt/1 && cp ./model.plan ensemble_hf/bert-trt/1/model.plan && rm -rf ./model.plan ./conversion_bs16_dy.txt ./model.onnx

## Option 2 - TensorRT Model with SageMaker Notebook Instances 

To make sure we use TensorRT version and dependencies that are compatible with the ones in our Triton container, we compile the model using the corresponding version of NVIDIA's PyTorch container image.

If you take a look at the python files within the `workspace` folder, you will see that we are first convert the model into ONNX format, specifying dynamic axis indexes so that inputs with a different batch size and sequence length can be passed to the model. TensorRT will treat other input dimensions as fixed, and optimize for those.


In [ ]:
! docker run --gpus=all --rm -it -v `pwd`/workspace:/workspace nvcr.io/nvidia/pytorch:21.08-py3 /bin/bash generate_model_trt.sh $model_id

In [ ]:
! rm -rf ensemble_hf/bert-trt && mkdir -p ensemble_hf/bert-trt/1 && cp workspace/model.plan ensemble_hf/bert-trt/1/model.plan && rm -rf workspace/model.onnx workspace/core*

Explore the output logs of the compilation process; at the very end, we get a section headlined "=== Performance summary ===" which gives us a series of metrics on the obtained engine's performance (latency, throughput, etc...). 

# Part 3 - Run Local Triton Inference Server

> **WARNING**: The cells under part 3 will only work if run within a SageMaker Notebook Instance!




The following cells run the Triton Inference Server container in the background and load all the models within the folder `/ensemble_hf`. The docker won't fail if one or more of the model fails because of `--exit-on-error=false`, which is useful for iterative code and model repository building. Remove `-d` to see the logs.

In [ ]:
!sudo docker system prune -f

In [ ]:
!docker run --gpus=all -d --shm-size=4G --rm -p8000:8000 -p8001:8001 -p8002:8002 -v$(pwd)/ensemble_hf:/model_repository nvcr.io/nvidia/tritonserver:21.08-py3 tritonserver --model-repository=/model_repository --exit-on-error=false --strict-model-config=false
time.sleep(20)

In [ ]:
CONTAINER_ID=!docker container ls -q
FIRST_CONTAINER_ID = CONTAINER_ID[0]

Uncomment the next cell and run it to view the container logs and understand Triton model loading.

In [ ]:
# !docker logs $FIRST_CONTAINER_ID -f

## Test TensorRT model by invoking the local Triton Server

In [ ]:
# Start a local Triton client
try:
    triton_client = http_client.InferenceServerClient(url="localhost:8000", verbose=True)
except Exception as e:
    print("context creation failed: " + str(e))
    sys.exit()

In [ ]:
# Create inputs to send to Triton
model_name = "ensemble"

text_inputs = ["Sentence 1", "Sentence 2"]

# Text is passed to Trtion as BYTES
inputs = []
inputs.append(http_client.InferInput("INPUT0", [len(text_inputs), 1], "BYTES"))

# We need to structure batch inputs as such
batch_request = [[text_inputs[i]] for i in range(len(text_inputs))]
input0_real = np.array(batch_request, dtype=np.object_)

inputs[0].set_data_from_numpy(input0_real, binary_data=False)

In [ ]:
outputs = []

outputs.append(http_client.InferRequestedOutput("finaloutput"))

In [ ]:
results = triton_client.infer(model_name=model_name, inputs=inputs, outputs=outputs)

In [ ]:
outputs_data = results.as_numpy("finaloutput")

for idx, output in enumerate(outputs_data):
    print(text_inputs[idx])
    print(output)

In [ ]:
# Use this to stop the container that was started in detached mode
!docker kill $FIRST_CONTAINER_ID

***

# Part 4 - Deploy Triton to SageMaker Real-Time Endpoint

## Deploy with SageMaker Triton container

First we get the URI for the Sagemaker Triton container image that matches the one we used for TensorRT model compilation.

In [ ]:
if region not in account_id_map.keys():
    raise ("UNSUPPORTED REGION")

base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"

triton_image_uri = "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:21.08-py3".format(
    account_id=account_id_map[region], region=region, base=base
)

triton_image_uri

In [ ]:
print(sagemaker_session.default_bucket())

In [ ]:
ensemble_prefix = "mme_gpu_tests/ensemble-singlemodel"
!tar -C ensemble_hf/ -czf ensemble-sentencetrans.tar.gz .
model_uri_tf = sagemaker_session.upload_data(
    path="ensemble-sentencetrans.tar.gz", key_prefix=ensemble_prefix
)

print("S3 model uri: {}".format(model_uri_tf))

In [ ]:
# Important to define what which one of the models loaded by Triton is the default to be served by SM
# That is, SAGEMAKER_TRITON_DEFAULT_MODEL_NAME
container_model = {
    "Image": triton_image_uri,
    "ModelDataUrl": model_uri_tf,
    "Mode": "SingleModel",
    "Environment": {"SAGEMAKER_TRITON_DEFAULT_MODEL_NAME": "ensemble"},
}

Register the model with Sagemaker.

In [ ]:
sm_model_name = "triton-sentence-ensemble" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

create_model_response = sm_client.create_model(
    ModelName=sm_model_name, ExecutionRoleArn=role, PrimaryContainer=container_model
)

print("Model Arn: " + create_model_response["ModelArn"])

Create an endpoint configuration.

In [ ]:
endpoint_config_name = "triton-sentence-ensemble" + time.strftime(
    "%Y-%m-%d-%H-%M-%S", time.gmtime()
)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.g4dn.xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Deploy the endpoint.

In [ ]:
endpoint_name = "triton-sentence-ensemble" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Wait for the endpoint to be up and running.

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

***

## Test the SageMaker Triton Endpoint

In [ ]:
text_inputs = ["Sentence 1", "Sentence 2"]

inputs = []
inputs.append(http_client.InferInput("INPUT0", [len(text_inputs), 1], "BYTES"))

batch_request = [[text_inputs[i]] for i in range(len(text_inputs))]

input0_real = np.array(batch_request, dtype=np.object_)

inputs[0].set_data_from_numpy(input0_real, binary_data=False)

len(input0_real)

In [ ]:
outputs = []

outputs.append(http_client.InferRequestedOutput("finaloutput"))

In [ ]:
request_body, header_length = http_client.InferenceServerClient.generate_request_body(
    inputs, outputs=outputs
)

print(request_body)

In [ ]:
response = sm_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/vnd.sagemaker-triton.binary+json;json-header-size={}".format(
        header_length
    ),
    Body=request_body,
)

In [ ]:
## json.loads fails
# a = json.loads(response["Body"].read().decode("utf8"))

header_length_prefix = "application/vnd.sagemaker-triton.binary+json;json-header-size="
header_length_str = response["ContentType"][len(header_length_prefix) :]

# Read response body
result = http_client.InferenceServerClient.parse_response_body(
    response["Body"].read(), header_length=int(header_length_str)
)

outputs_data = result.as_numpy("finaloutput")

for idx, output in enumerate(outputs_data):
    print(text_inputs[idx])
    print(output)